## Classifier

This file is to make a image classifier using AutoML (Neural Architecture Search) with Auto-Keras 

In [1]:
# import all relate modules
import tensorflow as tf
import math
import os
import numpy as np
import autokeras as ak # auto-keras for finding a image classifier 

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split

In [2]:
# define constant values
NUM_CLASSES = 18
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 16

In [3]:
# dataset path
dataset_path = '../datasets'

In [5]:
# define numeric class label
classes = {
    'call_me': 0,
    'clasp_hands': 1,
    'cross_arms': 2,
    'hide_one_arm': 3,
    'hide_two_arms': 4,
    'hold': 5,
    'list': 6,
    'move': 7,
    'open_one_arm': 8,
    'open_two_arms': 9,
    'point': 10,
    'roll': 11,
    'rotate_head': 12,
    'show_level': 13,
    'show_small_thing': 14,
    'stand_improperly': 15,
    'stand_properly': 16,
    'touch_body': 17
 }

## Get File List
getting all files from the directories that locate inside the given <code>root_dir</code> folder.

In [7]:
extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']


def get_file_list(root_dir):
    file_list = []
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(ext in filename for ext in extensions):
                file_list.append(os.path.join(root, filename))
    return file_list

Load and Preprocess the input images and set the label for each image

In [8]:
filenames = sorted(get_file_list(dataset_path))
X, y = [], np.zeros([len(filenames), 18])

i = 0
for fn in filenames:
    img = image.load_img(fn, target_size=(IMG_WIDTH, IMG_HEIGHT))
    x = image.img_to_array(img)
    X.append(x)
    y[i][classes[fn.split('/')[-2]]] = 1
    i += 1

Split train set and test set

In [12]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X), y, train_size=4180, test_size=500, stratify=y, random_state=7)

## Model Training
- Specify the auto-keras hyperparameter for searching a (sub) optimal image classifier. (max trials is 5 with accuracy metric).
- Set early stopiing when there is no improvement on validation accuracy with restoring best weights.
- Validation data is equal to 627 instances (15% of training set).

In [13]:
clf = ak.ImageClassifier(max_trials=5, metrics=['accuracy'])
es = tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_accuracy', restore_best_weights=True)

clf.fit(X_train, y_train, batch_size=BATCH_SIZE, validation_split=0.15, epochs=16, callbacks=[es], verbose=2)

Epoch 1/16
223/223 - 14s - loss: 3.4393 - accuracy: 0.3296 - val_loss: 1.1248 - val_accuracy: 0.6667
Epoch 2/16
223/223 - 13s - loss: 0.5547 - accuracy: 0.8302 - val_loss: 0.5837 - val_accuracy: 0.7941
Epoch 3/16
223/223 - 13s - loss: 0.1760 - accuracy: 0.9451 - val_loss: 0.5059 - val_accuracy: 0.8317
Epoch 4/16
223/223 - 13s - loss: 0.0675 - accuracy: 0.9779 - val_loss: 0.4889 - val_accuracy: 0.8382
Epoch 5/16
223/223 - 13s - loss: 0.0458 - accuracy: 0.9857 - val_loss: 0.4738 - val_accuracy: 0.8382
Epoch 6/16
223/223 - 13s - loss: 0.0226 - accuracy: 0.9944 - val_loss: 0.6164 - val_accuracy: 0.8317
Epoch 7/16
223/223 - 13s - loss: 0.0258 - accuracy: 0.9924 - val_loss: 0.5532 - val_accuracy: 0.8284
Epoch 8/16
223/223 - 13s - loss: 0.0143 - accuracy: 0.9964 - val_loss: 0.4547 - val_accuracy: 0.8562
Epoch 9/16
223/223 - 13s - loss: 0.0145 - accuracy: 0.9952 - val_loss: 0.5197 - val_accuracy: 0.8513
Epoch 10/16
223/223 - 13s - loss: 0.0159 - accuracy: 0.9964 - val_loss: 0.5175 - val_accura

Epoch 1/16
223/223 - 25s - loss: 3.0176 - accuracy: 0.0569 - val_loss: 6.8174 - val_accuracy: 0.0572
Epoch 2/16
223/223 - 23s - loss: 2.9307 - accuracy: 0.0591 - val_loss: 3.6389 - val_accuracy: 0.0605
Epoch 3/16
223/223 - 24s - loss: 2.9047 - accuracy: 0.0670 - val_loss: 2.9481 - val_accuracy: 0.0817
Epoch 4/16
223/223 - 24s - loss: 2.8861 - accuracy: 0.0774 - val_loss: 5.3447 - val_accuracy: 0.0931
Epoch 5/16
223/223 - 24s - loss: 2.8577 - accuracy: 0.0824 - val_loss: 5.9215 - val_accuracy: 0.0703
Epoch 6/16
223/223 - 23s - loss: 2.8433 - accuracy: 0.0804 - val_loss: 2.9394 - val_accuracy: 0.0915
Epoch 7/16
223/223 - 23s - loss: 2.8069 - accuracy: 0.0950 - val_loss: 2.8901 - val_accuracy: 0.1078
Epoch 8/16
223/223 - 23s - loss: 2.7788 - accuracy: 0.1118 - val_loss: 3.2974 - val_accuracy: 0.0882
Epoch 9/16
223/223 - 24s - loss: 2.7548 - accuracy: 0.1104 - val_loss: 2.8613 - val_accuracy: 0.1209
Epoch 10/16
223/223 - 23s - loss: 2.7317 - accuracy: 0.1197 - val_loss: 3.4689 - val_accura

Epoch 1/16
223/223 - 93s - loss: 30.6669 - accuracy: 0.0510 - val_loss: 2.8902 - val_accuracy: 0.0621
Epoch 2/16
223/223 - 93s - loss: 2.8902 - accuracy: 0.0558 - val_loss: 2.8905 - val_accuracy: 0.0621
Epoch 3/16
223/223 - 92s - loss: 2.8899 - accuracy: 0.0533 - val_loss: 2.8908 - val_accuracy: 0.0490
Epoch 4/16
223/223 - 92s - loss: 2.8897 - accuracy: 0.0586 - val_loss: 2.8910 - val_accuracy: 0.0490
Epoch 5/16
223/223 - 92s - loss: 2.8895 - accuracy: 0.0617 - val_loss: 2.8913 - val_accuracy: 0.0490
Epoch 6/16
223/223 - 92s - loss: 2.8894 - accuracy: 0.0617 - val_loss: 2.8916 - val_accuracy: 0.0490


Epoch 1/16
223/223 - 133s - loss: 3.4534 - accuracy: 0.0589 - val_loss: 2.9829 - val_accuracy: 0.0670
Epoch 2/16
223/223 - 135s - loss: 3.0205 - accuracy: 0.0597 - val_loss: 3.0580 - val_accuracy: 0.0686
Epoch 3/16
223/223 - 134s - loss: 2.9749 - accuracy: 0.0675 - val_loss: 3.0141 - val_accuracy: 0.0997
Epoch 4/16
223/223 - 135s - loss: 2.9012 - accuracy: 0.0821 - val_loss: 2.9170 - val_accuracy: 0.0817
Epoch 5/16
223/223 - 134s - loss: 2.8546 - accuracy: 0.0919 - val_loss: 2.9234 - val_accuracy: 0.1062
Epoch 6/16
223/223 - 134s - loss: 2.8255 - accuracy: 0.0922 - val_loss: 2.9647 - val_accuracy: 0.0784
Epoch 7/16
223/223 - 134s - loss: 2.7886 - accuracy: 0.0987 - val_loss: 2.8087 - val_accuracy: 0.1029
Epoch 8/16
223/223 - 133s - loss: 2.7617 - accuracy: 0.1096 - val_loss: 2.8632 - val_accuracy: 0.1160
Epoch 9/16
223/223 - 134s - loss: 2.7507 - accuracy: 0.1197 - val_loss: 2.9809 - val_accuracy: 0.0882
Epoch 10/16
223/223 - 135s - loss: 2.7376 - accuracy: 0.1065 - val_loss: 2.8039 - 

Epoch 1/16
223/223 - 30s - loss: 2.8348 - accuracy: 0.0869 - val_loss: 2.7270 - val_accuracy: 0.1683
Epoch 2/16
223/223 - 30s - loss: 2.5418 - accuracy: 0.2040 - val_loss: 2.3401 - val_accuracy: 0.2876
Epoch 3/16
223/223 - 30s - loss: 2.1377 - accuracy: 0.3436 - val_loss: 1.7994 - val_accuracy: 0.4624
Epoch 4/16
223/223 - 30s - loss: 1.4233 - accuracy: 0.5504 - val_loss: 1.3137 - val_accuracy: 0.6046
Epoch 5/16
223/223 - 30s - loss: 0.9792 - accuracy: 0.6914 - val_loss: 1.1414 - val_accuracy: 0.6569
Epoch 6/16
223/223 - 30s - loss: 0.7448 - accuracy: 0.7649 - val_loss: 1.0400 - val_accuracy: 0.6814
Epoch 7/16
223/223 - 31s - loss: 0.5843 - accuracy: 0.8094 - val_loss: 0.9249 - val_accuracy: 0.7222
Epoch 8/16
223/223 - 30s - loss: 0.4565 - accuracy: 0.8439 - val_loss: 0.8792 - val_accuracy: 0.7516
Epoch 9/16
223/223 - 30s - loss: 0.3667 - accuracy: 0.8792 - val_loss: 0.8055 - val_accuracy: 0.7729
Epoch 10/16
223/223 - 30s - loss: 0.2693 - accuracy: 0.9137 - val_loss: 0.8155 - val_accura

INFO:tensorflow:Oracle triggered exit
Epoch 1/16
262/262 - 9s - loss: 3.3148 - accuracy: 0.3883
Epoch 2/16
262/262 - 9s - loss: 0.4587 - accuracy: 0.8579
Epoch 3/16
262/262 - 9s - loss: 0.1220 - accuracy: 0.9617
Epoch 4/16
262/262 - 10s - loss: 0.0479 - accuracy: 0.9880
Epoch 5/16
262/262 - 9s - loss: 0.0258 - accuracy: 0.9935
Epoch 6/16
262/262 - 9s - loss: 0.0153 - accuracy: 0.9971
Epoch 7/16
262/262 - 10s - loss: 0.0157 - accuracy: 0.9957
Epoch 8/16
262/262 - 9s - loss: 0.0138 - accuracy: 0.9967
Epoch 9/16
262/262 - 9s - loss: 0.0090 - accuracy: 0.9978
Epoch 10/16
262/262 - 9s - loss: 0.0137 - accuracy: 0.9950
Epoch 11/16
262/262 - 9s - loss: 0.0105 - accuracy: 0.9969
Epoch 12/16
262/262 - 9s - loss: 0.0042 - accuracy: 0.9988
Epoch 13/16
262/262 - 9s - loss: 0.0083 - accuracy: 0.9974
Epoch 14/16
262/262 - 9s - loss: 0.0143 - accuracy: 0.9962
Epoch 15/16
262/262 - 9s - loss: 0.0219 - accuracy: 0.9931
Epoch 16/16
262/262 - 9s - loss: 0.0176 - accuracy: 0.9950


In [14]:
# Evaluate the test accuracy
clf.evaluate(X_test, y_test)

32/32 [==============================] - ETA: 0s - loss: 0.1801 - accuracy: 0.87 - ETA: 0s - loss: 0.4085 - accuracy: 0.87 - ETA: 0s - loss: 0.3294 - accuracy: 0.88 - ETA: 0s - loss: 0.3328 - accuracy: 0.89 - ETA: 0s - loss: 0.3142 - accuracy: 0.89 - ETA: 0s - loss: 0.3284 - accuracy: 0.88 - ETA: 0s - loss: 0.3372 - accuracy: 0.89 - ETA: 0s - loss: 0.3581 - accuracy: 0.88 - ETA: 0s - loss: 0.4458 - accuracy: 0.87 - ETA: 0s - loss: 0.4625 - accuracy: 0.88 - ETA: 0s - loss: 0.4665 - accuracy: 0.88 - 1s 19ms/step - loss: 0.4628 - accuracy: 0.8840


[0.4627579152584076, 0.8840000033378601]

## Export Best Model

In [15]:
best_model = clf.export_model()
best_model.save('gesture_model.h5')

## Test saved model
Test load saved model with summary function to see the model architecture.

In [16]:
from tensorflow.keras.models import load_model
loaded_model = load_model('gesture_model.h5')
loaded_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
normalization (Normalization (None, 224, 224, 3)       7         
_________________________________________________________________
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
flatten (Flatten)            (None, 774400)            0     